In [3]:
%pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(file_path):
    """Stores the data for the problem."""
    data = {}
    locations = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            city_id = int(parts[0])
            x, y = map(int, parts[1:])
            locations.append((x, y))

    data["locations"] = locations
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def create_distance_callback(data, manager):
    """Creates callback to return distance between points."""
    distances_ = {}
    index_manager_ = manager
    # precompute distance between location to have distance callback in O(1)
    for from_counter, from_node in enumerate(data["locations"]):
        distances_[from_counter] = {}
        for to_counter, to_node in enumerate(data["locations"]):
            if from_counter == to_counter:
                distances_[from_counter][to_counter] = 0
            else:
                distances_[from_counter][to_counter] = abs(
                    from_node[0] - to_node[0]
                ) + abs(from_node[1] - to_node[1])

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback


def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print(f"Objective: {assignment.ObjectiveValue()}")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    plan_output += f"Distance of the route: {route_distance}m\n"
    print(plan_output)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model("dataset.txt")

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["locations"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    distance_callback = create_distance_callback(data, manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(manager, routing, assignment)


main()



Objective: 43828
Route for vehicle 0:
 0 -> 7 -> 15 -> 21 -> 2 -> 33 -> 40 -> 28 -> 1 -> 41 -> 25 -> 3 -> 44 -> 34 -> 9 -> 23 -> 31 -> 38 -> 47 -> 4 -> 24 -> 13 -> 22 -> 10 -> 12 -> 20 -> 46 -> 19 -> 11 -> 39 -> 14 -> 32 -> 45 -> 35 -> 29 -> 42 -> 16 -> 26 -> 18 -> 36 -> 5 -> 27 -> 6 -> 17 -> 43 -> 30 -> 37 -> 8 -> 0
Distance of the route: 43828m



In [6]:
from ortools.linear_solver import pywraplp
import time

def solve_tsp(cost_matrix):
    num_cities = len(cost_matrix)
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return
    
    # Decision Variables
    x = {}
    for i in range(num_cities):
        for j in range(num_cities):
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')
    
    # Constraints
    for i in range(num_cities):
        solver.Add(sum(x[i, j] for j in range(num_cities)) == 1)  # Outgoing edges from city i
        solver.Add(sum(x[j, i] for j in range(num_cities)) == 1)  # Incoming edges to city i
    
    for i in range(num_cities):
        solver.Add(x[i, i] == 0)  # No self-loops
    
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                solver.Add(x[i, j] + x[j, i] <= 1)  # Ensure no duplicate paths between any pair of cities
    
    # Objective Function
    objective = solver.Objective()
    for i in range(num_cities):
        for j in range(num_cities):
            objective.SetCoefficient(x[i, j], cost_matrix[i][j])
    objective.SetMinimization()
    
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        tour = []
        next_city = 0
        total_cost = 0
        while True:
            tour.append(next_city)
            for j in range(num_cities):
                if x[next_city, j].solution_value() > 0:
                    total_cost += cost_matrix[next_city][j]  # Add cost from current city to next city to total cost
                    next_city = j
                    break
            if next_city == 0:
                break
        return tour, total_cost
    else:
        return None, None

def read_cost_matrix_from_file(file_path):
    with open(file_path, 'r') as file:
        num_cities = int(file.readline().strip('\ufeffï»¿').strip())
        cost_matrix = []
        for _ in range(num_cities):
            row = list(map(int, file.readline().strip().split()))
            cost_matrix.append(row)
    return cost_matrix

def main():
    file_path = r"C:\Users\admin\Documents\WorkSpace\TKPTGT\btl\TSP\TSP\dataset3.txt"
    cost_matrix = read_cost_matrix_from_file(file_path)
    
    optimal_tour, total_cost = solve_tsp(cost_matrix)
    if optimal_tour:
        print("Hành trình tối ưu:", optimal_tour)
        print("Tổng chi phí:", total_cost)
    else:
        print("Không tìm thấy hành trình tối ưu.")

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    execution_time = end_time - start_time
    print("Thời gian chạy:", execution_time, "giây")


Hành trình tối ưu: [0, 12, 1, 14, 8, 4, 6, 2, 11, 13, 9, 7, 5, 3, 10]
Tổng chi phí: 291
Thời gian chạy: 0.016916513442993164 giây
